In [9]:
# First, let's reload the data with proper analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Load the data
shale_gas = pd.read_csv('newdata.csv')

print("Dataset shape:", shale_gas.shape)
print("\nDataset columns:")
print(shale_gas.columns.tolist())
print("\nFirst 5 rows:")
print(shale_gas.head())
print("\nData types:")
print(shale_gas.dtypes)
print("\nBasic statistics:")
print(shale_gas.describe())

# Check for missing values
print("\nMissing values:")
print(shale_gas.isnull().sum())

# Check class distribution in Category
print("\nCategory distribution:")
print(shale_gas['Category'].value_counts())
print("\nCategory proportions:")
print(shale_gas['Category'].value_counts(normalize=True))

# Enhanced EDA function
def detailed_eda(df):
    print("="*80)
    print("ENHANCED EXPLORATORY DATA ANALYSIS")
    print("="*80)
    
    # 1. Target variable analysis
    print("\n1. TARGET VARIABLE (EUR) ANALYSIS:")
    print("-" * 40)
    print(f"Mean EUR: {df['EUR'].mean():.2f}")
    print(f"Median EUR: {df['EUR'].median():.2f}")
    print(f"Std EUR: {df['EUR'].std():.2f}")
    print(f"Min EUR: {df['EUR'].min():.2f}")
    print(f"Max EUR: {df['EUR'].max():.2f}")
    print(f"Range EUR: {df['EUR'].max() - df['EUR'].min():.2f}")
    
    # 2. Correlation with EUR
    print("\n2. TOP CORRELATIONS WITH EUR:")
    print("-" * 40)
    corr_matrix = df.select_dtypes(include=[np.number]).corr()
    eur_corr = corr_matrix['EUR'].sort_values(ascending=False)
    for feature, corr in eur_corr.items():
        if feature != 'EUR':
            print(f"{feature:20s}: {corr:7.3f}")
    
    # 3. Feature analysis by category
    print("\n3. FEATURE COMPARISON BY CATEGORY:")
    print("-" * 40)
    for col in df.select_dtypes(include=[np.number]).columns:
        if col != 'EUR':
            new_design_mean = df[df['Category'] == 'New_Design'][col].mean()
            vintage_mean = df[df['Category'] == 'Vintage'][col].mean()
            print(f"{col:20s}: New Design={new_design_mean:7.2f}, Vintage={vintage_mean:7.2f}, Diff={(new_design_mean-vintage_mean):7.2f}")
    
    return corr_matrix

corr_matrix = detailed_eda(shale_gas)

# Visualizations
plt.figure(figsize=(20, 15))

# 1. EUR distribution
plt.subplot(3, 3, 1)
sns.histplot(data=shale_gas, x='EUR', hue='Category', kde=True)
plt.title('EUR Distribution by Category')

# 2. Top correlated features with EUR
top_features = corr_matrix['EUR'].sort_values(ascending=False).index[1:6]
for i, feature in enumerate(top_features):
    plt.subplot(3, 3, i+2)
    sns.scatterplot(data=shale_gas, x=feature, y='EUR', hue='Category', alpha=0.6)
    plt.title(f'{feature} vs EUR')
    plt.tight_layout()

# 3. Correlation heatmap
plt.subplot(3, 3, 7)
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True)
plt.title('Correlation Matrix')

plt.tight_layout()
plt.show()

# Boxplots for important features by category
important_features = ['Lateral Length', 'Proppant Loading', 'Porosity', 'Injection Rate', 'Thickness']
plt.figure(figsize=(15, 10))
for i, feature in enumerate(important_features, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(data=shale_gas, x='Category', y=feature)
    plt.title(f'{feature} by Category')
plt.tight_layout()
plt.show()

# Data Preprocessing
def preprocess_data(df):
    df_processed = df.copy()
    
    # Encode categorical variable
    le = LabelEncoder()
    df_processed['Category_encoded'] = le.fit_transform(df_processed['Category'])
    
    # Drop the original categorical column
    df_processed = df_processed.drop('Category', axis=1)
    
    return df_processed, le

# Split data
shale_gas_processed, label_encoder = preprocess_data(shale_gas)

# Prepare features and target
X = shale_gas_processed.drop('EUR', axis=1)
y = shale_gas_processed['EUR']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=shale_gas['Category']
)

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    metrics = {
        'Model': model_name,
        'Train RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'Test RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'Train MAE': mean_absolute_error(y_train, y_train_pred),
        'Test MAE': mean_absolute_error(y_test, y_test_pred),
        'Train R2': r2_score(y_train, y_train_pred),
        'Test R2': r2_score(y_test, y_test_pred)
    }
    
    return metrics, y_test_pred, model

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate all models
results = []
predictions = {}
trained_models = {}

for name, model in models.items():
    metrics, y_pred, trained_model = evaluate_model(
        model, X_train_scaled, X_test_scaled, y_train, y_test, name
    )
    results.append(metrics)
    predictions[name] = y_pred
    trained_models[name] = trained_model

# Create results dataframe
results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(results_df.round(4))

# Feature importance analysis
print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

# Get feature importance from Random Forest
rf_model = trained_models['Random Forest']
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nRandom Forest Feature Importance:")
print(feature_importance)

# Visualization of feature importance
plt.figure(figsize=(12, 6))
sns.barplot(data=feature_importance, x='importance', y='feature', palette='viridis')
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

# Compare predictions vs actual
best_model_name = results_df.loc[results_df['Test R2'].idxmax(), 'Model']
print(f"\nBest model: {best_model_name}")

# Plot predictions vs actual for best model
y_pred_best = predictions[best_model_name]

plt.figure(figsize=(15, 5))

# Scatter plot
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred_best, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual EUR')
plt.ylabel('Predicted EUR')
plt.title(f'{best_model_name}: Predictions vs Actual')
plt.grid(True, alpha=0.3)

# Residual plot
plt.subplot(1, 3, 2)
residuals = y_test - y_pred_best
plt.scatter(y_pred_best, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted EUR')
plt.ylabel('Residuals')
plt.title(f'{best_model_name}: Residual Plot')
plt.grid(True, alpha=0.3)

# Distribution of errors
plt.subplot(1, 3, 3)
sns.histplot(residuals, kde=True)
plt.xlabel('Prediction Error')
plt.title(f'{best_model_name}: Error Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Hyperparameter tuning for the best model
if best_model_name in ['Random Forest', 'Gradient Boosting', 'XGBoost']:
    print(f"\nPerforming hyperparameter tuning for {best_model_name}...")
    
    if best_model_name == 'Random Forest':
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        model = RandomForestRegressor(random_state=42)
    
    elif best_model_name == 'Gradient Boosting':
        param_grid = {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 4, 5],
            'min_samples_split': [2, 5]
        }
        model = GradientBoostingRegressor(random_state=42)
    
    elif best_model_name == 'XGBoost':
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9, 1.0]
        }
        model = xgb.XGBRegressor(random_state=42)
    
    grid_search = GridSearchCV(
        model, param_grid, cv=5, 
        scoring='neg_mean_squared_error',
        n_jobs=-1, verbose=1
    )
    
    grid_search.fit(X_train_scaled, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {-grid_search.best_score_:.4f}")
    
    # Evaluate best model
    best_model = grid_search.best_estimator_
    y_pred_tuned = best_model.predict(X_test_scaled)
    
    print(f"\nTuned {best_model_name} Performance:")
    print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_tuned)):.4f}")
    print(f"Test MAE: {mean_absolute_error(y_test, y_pred_tuned):.4f}")
    print(f"Test R2: {r2_score(y_test, y_pred_tuned):.4f}")

# Feature engineering suggestions
print("\n" + "="*80)
print("FEATURE ENGINEERING SUGGESTIONS")
print("="*80)

# Create interaction features
X_train_engineered = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_engineered = pd.DataFrame(X_test_scaled, columns=X.columns)

# Suggest potential interaction features based on domain knowledge
interaction_suggestions = [
    ('Lateral Length', 'Proppant Loading'),
    ('Porosity', 'Thickness'),
    ('Injection Rate', 'Stage Spacing'),
    ('bbl/ft', 'Proppant Loading')
]

print("\nSuggested interaction features to try:")
for feat1, feat2 in interaction_suggestions:
    if feat1 in X.columns and feat2 in X.columns:
        print(f"- {feat1} × {feat2}")

# Business insights
print("\n" + "="*80)
print("BUSINESS INSIGHTS")
print("="*80)

# 1. EUR comparison between categories
new_design_eur = shale_gas[shale_gas['Category'] == 'New_Design']['EUR'].mean()
vintage_eur = shale_gas[shale_gas['Category'] == 'Vintage']['EUR'].mean()
print(f"1. Average EUR by category:")
print(f"   New Design: {new_design_eur:.2f}")
print(f"   Vintage: {vintage_eur:.2f}")
print(f"   Difference: {new_design_eur - vintage_eur:.2f} (+{(new_design_eur/vintage_eur - 1)*100:.1f}%)")

# 2. Key drivers analysis
print("\n2. Top 5 features influencing EUR:")
for i, row in feature_importance.head(5).iterrows():
    print(f"   {row['feature']}: {row['importance']:.3f}")

# 3. Optimal ranges for key features
print("\n3. Optimal ranges for top features in high-EUR wells:")
high_eur_threshold = shale_gas['EUR'].quantile(0.75)
high_eur_wells = shale_gas[shale_gas['EUR'] >= high_eur_threshold]

for feature in feature_importance.head(3)['feature'].tolist():
    if feature in shale_gas.columns:
        optimal_min = high_eur_wells[feature].min()
        optimal_max = high_eur_wells[feature].max()
        optimal_mean = high_eur_wells[feature].mean()
        print(f"   {feature}: {optimal_min:.1f} - {optimal_max:.1f} (avg: {optimal_mean:.1f})")

# Save the best model
import joblib
joblib.dump(trained_models[best_model_name], 'best_eur_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
print(f"\nBest model ({best_model_name}) saved to 'best_eur_model.pkl'")

FileNotFoundError: [Errno 2] No such file or directory: 'newdata.csv'